# Web Scraping con Python

Este script muestra algunas posibilidades de la biblioteca *Selenium* que nos permite obtener, automáticamente, datos de páginas web dinámicas, es decir que necesitan pulsar botones, etc, etc.

Selenium está pensado realmente para automatizar pruebas de entornos web, pero a nosotros nos servirá para nuestro propósito de hacer web scraping.

En particular, lo que vamos a hacer es arrancar google Chrome y manejarlo automáticamente desde Python.

Para ello necesitamos 3 cosas:

1.- Tener google Chrome

2.- Instalar la biblioteca selenium

3.- Tener un fichero controlador, un driver

Empezamos asegurarnos de que Selenium está instalado

In [ ]:
modules = ["selenium","chromedriver_autoinstaller"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user",  m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)  

Ahora abrimos el navegador

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url = 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCBusqueda.aspx'
driver.get(url)



Simulamos un click en la página de aceptar cookies


In [ ]:

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
#aceptar las cookies

try:
    cookies = driver.find_element(By.LINK_TEXT, "Aceptar cookies")
    cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se encontró el botón de aceptar cookies o ocurrió un error:", e)

# Hacer clic en la pestaña de coordenadas
try:
    coordenadas_tab = driver.find_element(By.LINK_TEXT, "COORDENADAS")
    coordenadas_tab.click()
    print("Se ha accedido a la pestaña de coordenadas.")
except Exception as e:
    print("No se encontró la pestaña de coordenadas o ocurrió un error:")
#coord.click()

datos de Coordenadas

In [ ]:
#lat = driver.find_element_by_id("ctl00_Contenido_txtLatitud")
#lon = driver.find_element_by_id("ctl00_Contenido_txtLongitud")
lat = driver.find_element(By.ID, "ctl00_Contenido_txtLatitud")

lon = driver.find_element(By.ID, "ctl00_Contenido_txtLongitud")
latitud  = "28.2723368"
longitud = "-16.64268"
lat.send_keys(latitud)
lon.send_keys(longitud)

Pulsar Botón para acceder a los datos concretos

In [ ]:
datos = driver.find_element(By.ID, "ctl00_Contenido_btnDatos")
datos.click()



In [ ]:
id = "ctl00_Contenido_tblInmueble"
#div = driver.find_element("id")
labels = driver.find_elements(By.XPATH, "//label")
for i,l in enumerate(labels):
    print(i,l.text)

Referencia catastral

In [ ]:
id = "ctl00_Contenido_tblInmueble"
pos = [0,3,4,5,7,8]
div = driver.find_element(By.ID,id)
labels = div.find_elements(By.XPATH,"//label")
for i,label in enumerate(labels):
    if i in pos:
        print(label.text)

Todo Junto

In [ ]:
import pandas as pd
file = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/catastro.csv"
df = pd.read_csv(file)
df

In [ ]:
from time import sleep
df = pd.read_csv(file)
coldf = len(df.columns)
url = 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCBusqueda.aspx'
# mejor con una lista de lsitas, pero así se entiende mejor
pos = [0,3,4,5,7,8]
nombres = ["ref","loc","clase","uso","superficie","año"]
df[nombres] = None
filenum = 0
for row in df.iterrows():
    # los datos están en row[1], row[0] es el índice
    fila = row[1]
    driver.get(url)
    coord   = driver.find_element(By.LINK_TEXT,"COORDENADAS")
    coord.click()
    sleep(0.5)
    # coordenadas
    lat = driver.find_element(By.ID,"ctl00_Contenido_txtLatitud")
    lon = driver.find_element(By.ID,"ctl00_Contenido_txtLongitud")
    latitud  = str(fila["lat"])
    longitud = str(fila["long"])
    lat.send_keys(latitud)
    lon.send_keys(longitud)
    sleep(0.25)
    # pulsar el botón, esto puede tardar
    datos = driver.find_element(By.ID,"ctl00_Contenido_btnDatos")
    datos.click()

    sleep(1.5)
    id = "ctl00_Contenido_tblInmueble"
    pos = [0,3,4,5,7,8]
    div = driver.find_elements(By.ID,id)
    if len(div)==1:
        div = div[0]
        labels = div.find_elements(By.XPATH,"//label")
        for i,label in enumerate(labels):
            if i in pos:
                df.iloc[filenum,coldf+pos.index(i)] = label.text
    filenum+=1



In [ ]:
df


### Más pruebas

Texto de la página

In [ ]:
html = driver.find_element(By.XPATH,"/html")
print(html.text)

Caminos absolutos

In [ ]:
head = driver.find_element(By.XPATH,"/html/head")
body = driver.find_element(By.XPATH,"/html/body")
html2 = body.find_element(By.XPATH,"/html")

In [ ]:
print(body.text)

Hijos de un elemento

In [ ]:
hijos = driver.find_elements(By.XPATH,"/html/body/*")
for element in hijos:
  print(element.tag_name)

Camino relativo

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body/*/div")
print(len(divs))

In [ ]:
divs = body.find_elements(By.XPATH,"./*/div")
print(len(divs))

Saltar pasos intermedios

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body//div")
print(len(divs))

In [ ]:
labels = driver.find_elements(By.XPATH,"//label")
print(len(labels))

Insertar texto

In [ ]:
lat = driver.find_element(By.ID,"ctl00_Contenido_txtLatitud")
lon = driver.find_element(By.ID,"ctl00_Contenido_txtLongitud")
latitud  = "28.2723368"
longitud = "-16.64268"
lat.send_keys(latitud)
lon.send_keys(longitud)



In [ ]:
datos = driver.find_element(By.ID,"ctl00_Contenido_btnDatos")
datos.click()

In [ ]:
id = "ctl00_Contenido_tblInmueble"
div = driver.find_element(By.ID,id)
label = div.find_element(By.XPATH,"//label")
print(label.text)

Otra forma, por texto

In [ ]:
xpath = "//*[./span/text()='Referencia catastral']//label"
etiqs = driver.find_element(By.XPATH,xpath)
print(etiqs.text)

Algunos ejemplos más....

In [ ]:
clase = driver.find_elements(By.XPATH,"(//label)[position()=3]")
print(clase[0].text)

etiqs = driver.find_elements(By.XPATH,"//label")
print(etiqs[2].text)

ulti = driver.find_elements(By.XPATH,"(//label)[last()]")
print(ulti[0].text)


Un pequeño ejemplo adicional, información metereológica en Madrid

In [ ]:
url2 = 'http://www.aemet.es/es/eltiempo/prediccion/municipios/madrid-id28079'
driver.get(url2)


In [ ]:
th = driver.find_elements(By.XPATH,"/html/body//*/th")
print(len(th))
th = driver.find_elements(By.XPATH,"/html/body//*/th[@class='borde_izq_dcha_estado_cielo no_wrap']")
print(len(th))
for e in th:
    print(e.text)

In [ ]:
driver.close()

Por Rafael Caballero. Del libro "Big data con Python". Gracias a José Ramón Guerra por las actualizaciones